# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN

# Read the CSV into DataFrame

In [4]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
#df = pd.get_dummies(df)
df.head(5)

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [5]:
# Create our features
X = df.copy()
X.drop("loan_status", axis=1, inplace=True)
X = pd.get_dummies(X)
#X.head(5)

# Create our target
target = ['loan_status']
y = df.loc[:,target].copy()
y.head()

,loan_status
0,low_risk
1,low_risk
2,low_risk
3,low_risk
4,low_risk


In [6]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [7]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [8]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [11]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [12]:
model_lr = LogisticRegression(solver='lbfgs', random_state=1)
model_lr.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [13]:
# Calculated the balanced accuracy score
pred_lr = model_lr.predict(X_test_scaled)
acc_score_lr = balanced_accuracy_score(y_test, pred_lr)

In [14]:
# Display the confusion matrix
cm_lr = confusion_matrix(y_test, pred_lr)
cm_lr_df = pd.DataFrame(
    cm_lr, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [15]:
# Print the imbalanced classification report
print("Confusion Matrix")
display(cm_lr_df)
print(f"Accuracy Score : {acc_score_lr}")
print("Classification Report")
print(classification_report_imbalanced(y_test, pred_lr))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,599,1
Actual 1,7031,11753


Accuracy Score : 0.8120127058489495
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      1.00      0.63      0.15      0.79      0.65       600
   low_risk       1.00      0.63      1.00      0.77      0.79      0.60     18784

avg / total       0.97      0.64      0.99      0.75      0.79      0.60     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [16]:
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'loan_status': 1})

In [17]:
# Train the Logistic Regression model using the resampled data
model_ro = LogisticRegression(solver='lbfgs', random_state=1)
model_ro.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [18]:
# Calculated the balanced accuracy score
pred_ro = model_ro.predict(X_test_scaled)
acc_score_ro = balanced_accuracy_score(y_test, pred_ro)

In [19]:
# Display the confusion matrix
cm_ro = confusion_matrix(y_test, pred_ro)
cm_ro_df = pd.DataFrame(
    cm_ro, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [20]:
# Print the imbalanced classification report
print("Confusion Matrix")
display(cm_ro_df)
print(f"Accuracy Score : {acc_score_ro}")
print("Classification Report")
print(classification_report_imbalanced(y_test, pred_ro))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,598,2
Actual 1,118,18666


Accuracy Score : 0.9951923622941511
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      1.00      0.99       600
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18784

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



### SMOTE Oversampling

In [21]:
# Resample the training data with SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'loan_status': 1})

In [22]:
# Train the Logistic Regression model using the resampled data
model_sm = LogisticRegression(solver='lbfgs', random_state=1)
model_sm.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [23]:
# Calculated the balanced accuracy score
pred_sm = model_sm.predict(X_test_scaled)
acc_score_sm = balanced_accuracy_score(y_test, pred_sm)

In [24]:
# Display the confusion matrix
cm_sm = confusion_matrix(y_test, pred_sm)
cm_sm_df = pd.DataFrame(
    cm_sm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [25]:
# Print the imbalanced classification report
print("Confusion Matrix")
display(cm_sm_df)
print(f"Accuracy Score : {acc_score_sm}")
print("Classification Report")
print(classification_report_imbalanced(y_test, pred_sm))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,598,2
Actual 1,118,18666


Accuracy Score : 0.9951923622941511
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      1.00      0.99       600
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18784

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [26]:
# Resample the data using the ClusterCentroids resampler
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'loan_status': 1})

In [27]:
# Train the Logistic Regression model using the resampled data
model_us = LogisticRegression(solver='lbfgs', random_state=1)
model_us.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [28]:
# Calculated the balanced accuracy score
pred_us = model_us.predict(X_test_scaled)
acc_score_us = balanced_accuracy_score(y_test, pred_us)

In [29]:
# Display the confusion matrix
cm_us = confusion_matrix(y_test, pred_us)
cm_us_df = pd.DataFrame(
    cm_us, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [30]:
# Print the imbalanced classification report
print("Confusion Matrix")
display(cm_us_df)
print(f"Accuracy Score : {acc_score_us}")
print("Classification Report")
print(classification_report_imbalanced(y_test, pred_us))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,595,5
Actual 1,111,18673


Accuracy Score : 0.992878691084611
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       600
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18784

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [31]:
# Resample the training data with SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'loan_status': 1})

In [32]:
# Train the Logistic Regression model using the resampled data
model_uo = LogisticRegression(solver='lbfgs', random_state=1)
model_uo.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [33]:
# Calculated the balanced accuracy score
pred_uo = model_uo.predict(X_test_scaled)
acc_score_uo = balanced_accuracy_score(y_test, pred_uo)

In [34]:
# Display the confusion matrix
cm_uo = confusion_matrix(y_test, pred_uo)
cm_uo_df = pd.DataFrame(
    cm_uo, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [35]:
# Print the imbalanced classification report
print("Confusion Matrix")
display(cm_uo_df)
print(f"Accuracy Score : {acc_score_uo}")
print("Classification Report")
print(classification_report_imbalanced(y_test, pred_uo))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,598,2
Actual 1,119,18665


Accuracy Score : 0.9951657438955139
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      1.00      0.99      0.91      1.00      0.99       600
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18784

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   The answer here is mixed really.  The simple linear regression didn't perform well and also had the lowest accuracy score among all the models tested.  When looking at the other models it is really balanced as they all seemingly had very strong balanced accuracy score of around 99% so any would be good choices.

2. Which model had the best recall score?

    Similar story here - given that the accruacy score above for the simple regression model - I would say that the other models tested should be used even though the recall score for all models was very similar in testing.

3. Which model had the best geometric mean score?

    Again based on reading the models all perform very similarly here with a geo socre near 1, that said again the simple linear model given low percision and recall, I would suggest using one of the final models for prediction of high/low risk going forward.
